In [ ]:
import sys

sys.path.append('..')

In [ ]:
import torch

data = torch.load('../processed_data/complete_dataset/processed_data.pt')
label = torch.load('../processed_data/complete_dataset/processed_label.pt')

In [ ]:
from audio_toolbox.metrics import audio_dataset_split

RANDOM_STATE = 42
X_train, y_train, X_val, y_val,\
X_test, y_test = audio_dataset_split(data, label, train_val_test_ratio=(0.9, 0.05, 0.05), random_state=RANDOM_STATE)

In [ ]:
X_train_flat = X_train.view(X_train.shape[0], -1)
X_val_flat = X_val.view(X_val.shape[0], -1)
X_test_flat = X_test.view(X_test.shape[0], -1)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
pca = PCA(n_components=0.9)

train_scaled = scaler.fit_transform(X_train_flat)
val_scaled = scaler.transform(X_val_flat)
test_scaled = scaler.transform(X_test_flat)

device = 'cuda' if torch.cuda.is_available() else 'cpu' 
train_data = torch.tensor(pca.fit_transform(train_scaled), dtype=torch.float, device=device)
val_data = torch.tensor(pca.transform(val_scaled), dtype=torch.float, device=device)
test_data = torch.tensor(pca.transform(test_scaled), dtype=torch.float, device=device)

In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
y_train = torch.tensor(lb.fit_transform(y_train), device=device)
y_val = torch.tensor(lb.fit_transform(y_val), device=device)
y_test = torch.tensor(lb.fit_transform(y_test), device=device)

In [ ]:
from torch.utils.data import TensorDataset

datasets = {
    'train': TensorDataset(train_data, y_train),
    'val': TensorDataset(val_data, y_val),
    'test': TensorDataset(test_data, y_test)
}

In [ ]:
from audio_toolbox.models import LinearModel

input_size = train_data.size(1)
output_size = 10
batch_size = 32

model = LinearModel(input_size, output_size, hidden_dim=64, dropout_prob=0.8, device='cuda')


In [ ]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch import nn
from audio_toolbox.models import OneHotCrossEntropyLoss

loss_fn = OneHotCrossEntropyLoss()
learning_rate = 1e-3  # Adjust the learning rate as needed
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [ ]:
from audio_toolbox.trainer import ModelTrainer

trainer = ModelTrainer(datasets, model, loss_fn, optimizer, scheduler)

In [ ]:
trainer_config = {
    'save': False,
    'num_epochs': 100,
    'batch_size': batch_size,
    'metric': 'acc'
}

In [ ]:
trainer.train(**trainer_config)

In [ ]:
train_res = trainer.predict(train_data)
val_res = trainer.predict(val_data)
test_res = trainer.predict(test_data)

In [ ]:
from sklearn.metrics import accuracy_score

print(f"Train accuracy: {100 * accuracy_score(train_res, y_train):.2f}%")
print(f"Validation accuracy: {100 * accuracy_score(val_res, y_val):.2f}%")
print(f"Test accuracy: {100 * accuracy_score(test_res, y_test):.2f}%")

In [ ]:
from audio_toolbox.metrics import precision_recall

_, _, _, f1_train = precision_recall(trainer, train_data, y_train)
_, _, _, f1_val = precision_recall(trainer, val_data, y_val)
_, _, _, f1_test = precision_recall(trainer, test_data, y_test)

In [ ]:
print(f"Train f1 score: {f1_train:.4f}")
print(f"Validation f1 score: {f1_val:.4f}")
print(f"Test f1 score: {f1_test:.4f}")